#### Imports

In [84]:
import os
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Get data from .txt files

In [2]:
directory_path = '../data/jobs_html/'
files = os.listdir(directory_path)
count = 0
for i in files:
    count += 1
count

295

In [93]:
items = []

for file_name in files:
    file_path = os.path.join(directory_path, file_name)
    row = []
    job_id = file_name.replace('.txt', '')
    row.append(job_id)
    
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            soup = BeautifulSoup(file.read(), 'html.parser')
            container = soup.find_all('div', {'class': 'jobs-unified-top-card'})[0]
            title = container.find_all('h1', {'class': 'job-details-jobs-unified-top-card__job-title'})[0].getText().strip()
            row.append(title)
            company_name = container.find_all('a', {'class': 'app-aware-link'})[1].getText().strip()
            row.append(company_name)
            state = container.find(string = lambda text: text and '· ' in text)
            row.append(state)
            line = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[0].getText().strip()
            line = line.split('\n')
            line = [i for i in line if i != '']
            line = [i for i in line if i != ' ']
            if len(line) == 4:
                salary_range = line[0]
                remote = line[1]
                employment_type = line[2]
                experience_level = line[3]
                company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
            elif len(line) == 3:
                if '$' in line[0]:
                    salary_range = line[0]
                    remote = np.nan
                else:
                    salary_range = np.nan
                    remote = line[0]
                employment_type = line[1]
                experience_level = line[2]
                company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
            elif len(line) == 2:
                salary_range = np.nan
                remote = line[0]
                employment_type = line[1]
                experience_level = np.nan
                company_size = container.find_all('li', {'class': 'job-details-jobs-unified-top-card__job-insight'})[1].find('span').getText().strip()
            row.append(salary_range)
            row.append(remote)
            row.append(employment_type)
            row.append(experience_level)
            row.append(company_size)
            items.append(row)

columns = ['job_id', 'job_title', 'company_name', 'company_state', 'salary_range', 'remote_ratio', 'employment_type', 'experience_level', 'company_size']

linkedin = pd.DataFrame(items, columns=columns)
linkedin.sample(5)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
169,3769049759,Data Entry Clerk - Fully Remote,Data Nomad,· Estados Unidos,NaN,En remoto,Jornada completa,Sin experiencia,De 51 a 200 empleados · Servicios y tecnología...
175,3770685212,Data Engineer - Need candidates from Texas ( ...,Pulivarthi Group (PG),· Estados Unidos,NaN,En remoto,Jornada completa,Intermedio,De 51 a 200 empleados · Dotación y selección d...
150,3768286533,Summer Intern - Publishing Ethics and IPR,IEEE,"· Municipio de Piscataway, NJ",NaN,Presencial,Prácticas,Prácticas,Más de 10.001 empleados · Organizaciones sin á...
289,3775819495,"Staff Data Scientist, Core Experience Analytics","Reddit, Inc.",· Estados Unidos,198.200 $/año - 297.300 $/año,En remoto,Jornada completa,Sin experiencia,De 501 a 1.000 empleados · Desarrollo de software
63,3757898670,Senior Data Scientist,Acceler8 Talent,"· Washington, DC",NaN,En remoto,Jornada completa,Intermedio,De 11 a 50 empleados · Dotación y selección de...


In [96]:
linkedin.to_csv('../data/linkedin.csv', index=False)